## GPU Puzzles in CUDA C++
By Devin Shah - [@devinshah16](https://twitter.com/DevinShah16)

Puzzles adapted from [Sasha Rush](http://rush-nlp.com/)

GPUs are pretty cool.

This notebook is a bit more of an advanced attempt to teach GPU programming interactively. Instead of using Python bindings (through Numba), we will be directly working with CUDA C++ bindings. In this notebook, we will just be focusing on the kernels, but in a later video, I will walk through how to instantiate the kernels, which is a bit harder than using Numba's built in executor.

I recommend doing Sasha's notebook first, as the visualization are much clearer and will help build intuition.
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/srush/GPU-Puzzles/blob/main/GPU_puzzlers.ipynb)

Make your own copy of this notebook in Colab, turn on GPU mode in the settings (`Runtime / Change runtime type`, then set `Hardware accelerator` to `GPU`), and
then get to coding.

Read the [CUDA C++ bindings guide ](https://docs.nvidia.com/cuda/pdf/CUDA_C_Programming_Guide.pdf)

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


## Puzzle 2 - Zip
Implement a kernel that adds together each position of `a` and `b` and stores it in `out`. You have 1 thread per position.

In [ ]:
%%writefile zip.cu
#include <iostream>
#include <cassert>

__global__ void VecAdd(float* A, float* B, float* C) {
  int i = threadIdx.x;
  C[i] = A[i] + B[i];
}

int main() {
    const int N = 3;
    float A[N], B[N], C[N];

    for (int i = 0; i < N; i++) {
        A[i] = static_cast<float>(i);
        B[i] = static_cast<float>(N - i);
    }

    float *d_A, *d_B, *d_C;

    cudaMalloc(&d_A, sizeof(float) * N);
    cudaMalloc(&d_B, sizeof(float) * N);
    cudaMalloc(&d_C, sizeof(float) * N);

    cudaMemcpy(d_A, A, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, N * sizeof(float), cudaMemcpyHostToDevice);

    VecAdd<<<1, N>>>(d_A, d_B, d_C);

    cudaMemcpy(C, d_C, N * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    for (int i = 0; i < N; i++) {
      assert(C[i] == A[i] + B[i]);
    }

    std::cout << "Vector addition successful!" << std::endl;

    return 0;
}

Writing zip.cu


In [ ]:
!nvcc zip.cu -o zip
!./zip

Vector addition successful!


In [ ]:
%%writefile map.cu
#include <iostream>
#include <cassert>

__global__ void ScalarAdd(float* A, float* C) {
  int i = threadIdx.x;
  C[i] = A[i] + 10;
}

int main() {
  const int N = 3;
  float A[N], C[N];

  for (int i = 0; i < N; i++) {
    A[i] = static_cast<float>(i);
  }

  float *d_A, *d_C;

  cudaMalloc(&d_A, N * sizeof(float));
  cudaMalloc(&d_C, N * sizeof(float));

  cudaMemcpy(d_A, A, N * sizeof(float), cudaMemcpyHostToDevice);

  ScalarAdd<<<1, N>>>(d_A, d_C);

  cudaMemcpy(C, d_C, N * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_C);

  for (int i = 0; i < N; i++) {
    assert(C[i] == A[i] + 10);
  }

  std::cout << "Scalar addition is successful!" << std::endl;

  return 0;

}

Writing map.cu


In [ ]:
!nvcc map.cu -o map
!./map

Scalar addition is successful!


In [ ]:
%%writefile guards.cu
#include <iostream>
#include <cassert>

__global__ void Guards(float* A, float* C, float size) {
  int i = threadIdx.x;
  if (i < size) {
    C[i] = A[i] + 10;
  }
}

int main() {
  const int size = 3;
  float A[size], C[size];

  for (int i = 0; i < size; i++) {
    A[i] = static_cast<float>(i);
  }

  float *d_A, *d_C;

  cudaMalloc(&d_A, size * sizeof(float));
  cudaMalloc(&d_C, size * sizeof(float));

  cudaMemcpy(d_A, A, size * sizeof(float), cudaMemcpyHostToDevice);

  Guards<<<1, 10>>>(d_A, d_C, size);

  cudaMemcpy(C, d_C, size * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_C);

  for (int i = 0; i < size; i++){
    assert(C[i] == A[i] + 10);
  }

  std::cout << "Guards successful!" << std::endl;

  return 0;

}


Writing guards.cu


In [ ]:
!nvcc guards.cu -o guards
!./guards
!compute-sanitizer ./guards

Guards successful!
========= COMPUTE-SANITIZER
Guards successful!
========= ERROR SUMMARY: 0 errors


In [ ]:
%%writefile map_2d.cu

#include <iostream>
#include <cassert>

__global__ void Map2D(float* A, float* C, float size) {
  int local_i = threadIdx.x;
  int local_j = threadIdx.y;

  int index = local_i * size + local_j;
  if (local_i < size && local_j < size) {
    C[index] = A[index] + 10;
  }
}

int main() {

  const int size = 4;
  float A[size][size], C[size][size];

  for (int i = 0; i < size; i++) {
    for (int j = 0; j < size; j++) {
      A[i][j] = static_cast<float>(i) + static_cast<float>(j);
    }
  }

  float *d_A, *d_C;

  cudaMalloc(&d_A, (size * size) * sizeof(float));
  cudaMalloc(&d_C, (size * size) * sizeof(float));

  dim3 blockDim(size, size);

  cudaMemcpy(d_A, A, (size * size) * sizeof(float), cudaMemcpyHostToDevice);

  Map2D<<<1, blockDim>>>(d_A, d_C, size);

  cudaMemcpy(C, d_C, (size * size) * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_C);

  for (int i = 0; i < size; i++) {
    for (int j = 0; j < size; j++) {
      assert(C[i][j] == A[i][j] + 10);
    }
  }

  std::cout << "2D mapping successful" << std::endl;
  return 0;

}

Writing map_2d.cu


In [ ]:
!nvcc map_2d.cu -o map_2d
!./map_2d
!compute-sanitizer ./map_2d

2D mapping successful
========= COMPUTE-SANITIZER
2D mapping successful
========= ERROR SUMMARY: 0 errors


In [ ]:
%%writefile broadcast.cu

#include <iostream>
#include <cassert>

__global__ void Broadcast(float* A, float* B, float* C, int size) {
  int local_i = threadIdx.x;
  int local_j = threadIdx.y;

  int index = local_i * size + local_j;
  if (local_i < size && local_j < size) {
    C[index] = A[local_i] + B[local_j];
  }
}

int main() {

  const int size = 4;
  float A[size][1], B[1][size], C[size][size];

  for (int i = 0; i < size; i++) {
    A[i][0] = static_cast<float>(i);
  }

  for (int j = 0; j < size; j++) {
    B[0][j] = static_cast<float>(j);
  }

  float *d_A, *d_B, *d_C;

  cudaMalloc(&d_A, size * sizeof(float));
  cudaMalloc(&d_B, size * sizeof(float));
  cudaMalloc(&d_C, (size * size) * sizeof(float));

  cudaMemcpy(d_A, A, size * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, B, size * sizeof(float), cudaMemcpyHostToDevice);

  dim3 blockDim(size, size);

  Broadcast<<<1, blockDim>>>(d_A, d_B, d_C, size);

  cudaMemcpy(C, d_C, (size * size) * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  for (int i = 0; i < size; i++) {
    for (int j = 0; j < size; j++) {
      assert(C[i][j] == A[i][0] + B[0][j]);
    }
  }

  std::cout << "Broadcast successful" << std::endl;
  return 0;

}

Writing broadcast.cu


In [ ]:
!nvcc broadcast.cu -o broadcast
!./broadcast
!compute-sanitizer ./broadcast

Broadcast successful
========= COMPUTE-SANITIZER
Broadcast successful
========= ERROR SUMMARY: 0 errors


In [ ]:
%%writefile blocks.cu

#include <iostream>
#include <cassert>

__global__ void Blocks(float* A, float* C, float size) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < size) {
    C[i] = A[i] + 10;
  }
}

int main() {
  const int size = 5;
  float A[size], C[size];

  for (int i = 0; i < size; i++) {
    A[i] = static_cast<float>(i);
  }

  float *d_A, *d_C;

  cudaMalloc(&d_A, size * sizeof(float));
  cudaMalloc(&d_C, size * sizeof(float));

  cudaMemcpy(d_A, A, size * sizeof(float), cudaMemcpyHostToDevice);

  int threadsPerBlock = size - 1;
  int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;

  Blocks<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_C, size);

  cudaMemcpy(C, d_C, size * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_C);

  for (int i = 0; i < size; i++) {
    assert(C[i] == A[i] + 10);
  }

  std::cout << "Blocks successful!" << std::endl;
  return 0;

}

Writing blocks.cu


In [ ]:
!nvcc blocks.cu -o blocks
!./blocks
!compute-sanitizer ./blocks

Blocks successful!
========= COMPUTE-SANITIZER
Blocks successful!
========= ERROR SUMMARY: 0 errors


In [ ]:
%%writefile map2d_block.cu

#include <iostream>
#include <cassert>

__global__ void Map2DBlock(float* A, float* C, float size) {
  int local_i = blockDim.x * blockIdx.x + threadIdx.x;
  int local_j = blockDim.y * blockIdx.y + threadIdx.y;

  int index = local_i * size + local_j;

  if (local_i < size && local_j < size) {
    C[index] = A[index] + 10;
  }
}

int main() {

  const int size = 6;
  float A[size][size], C[size][size];

  for (int i = 0; i < size; i++) {
    for (int j = 0; j < size; j++) {
      A[i][j] = static_cast<float>(i) + static_cast<float>(j);
    }
  }

  float *d_A, *d_C;

  cudaMalloc(&d_A, (size * size) * sizeof(float));
  cudaMalloc(&d_C, (size * size) * sizeof(float));

  dim3 threadsPerBlock(size - 1, size - 1);
  dim3 blocksPerGrid(((size + threadsPerBlock.x - 1) / threadsPerBlock.x),
                    (((size + threadsPerBlock.y - 1) / threadsPerBlock.y)));

  cudaMemcpy(d_A, A, (size * size) * sizeof(float), cudaMemcpyHostToDevice);

  Map2DBlock<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_C, size);

  cudaMemcpy(C, d_C, (size * size) * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_C);

  for (int i = 0; i < size; i++) {
    for (int j = 0; j < size; j++) {
      assert(C[i][j] == A[i][j] + 10);
    }
  }

  std::cout << "2D mapping successful" << std::endl;
  return 0;

}

Writing map2d_block.cu


In [ ]:
!nvcc map2d_block.cu -o map2d_block
!./map2d_block
!compute-sanitizer ./map2d_block

2D mapping successful
========= COMPUTE-SANITIZER
2D mapping successful
========= ERROR SUMMARY: 0 errors


In [ ]:
%%writefile shared.cu

#include <iostream>
#include <cassert>

__global__ void Shared(float* A, float* C, float size) {
  extern __shared__ float sharedMem[];

  int i = blockDim.x * blockIdx.x + threadIdx.x;
  int local_i = threadIdx.x;

  if (i < size) {
    sharedMem[local_i] = A[i];
  }
  __syncthreads();

  if (i < size) {
    C[i] = sharedMem[local_i] + 10;
  }

}

int main() {
  const int size = 5;
  float A[size], C[size];

  for (int i = 0; i < size; i++) {
    A[i] = static_cast<float>(i);
  }

  float *d_A, *d_C;

  cudaMalloc(&d_A, size * sizeof(float));
  cudaMalloc(&d_C, size * sizeof(float));

  cudaMemcpy(d_A, A, size * sizeof(float), cudaMemcpyHostToDevice);

  int threadsPerBlock = size - 1;
  int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;
  int shared_size = threadsPerBlock * sizeof(float);

  Shared<<<blocksPerGrid, threadsPerBlock, shared_size>>>(d_A, d_C, size);

  cudaMemcpy(C, d_C, size * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_C);

  for (int i = 0; i < size; i++) {
    assert(C[i] == A[i] + 10);
  }

  std::cout << "Shared successful!" << std::endl;
  return 0;

}

Writing shared.cu


In [ ]:
!nvcc shared.cu -o shared
!./shared
!compute-sanitizer ./shared

Shared successful!
========= COMPUTE-SANITIZER
Shared successful!
========= ERROR SUMMARY: 0 errors


In [ ]:
%%writefile pooling.cu

#include <iostream>
#include <cassert>

__global__ void Pooling(float* A, float* C, float size) {
  extern __shared__ float sharedMem[];
  int i = blockDim.x * blockIdx.x + threadIdx.x;
  int local_i = threadIdx.x;

  if (i < size) {
    sharedMem[local_i] = A[i];
  }
  __syncthreads();

  if (local_i - 2 >= 0 && i < size) {
    C[i] = sharedMem[local_i - 2] + sharedMem[local_i - 1] + sharedMem[local_i];
  }
}

int main() {
  const int size = 4;
  float A[size], C[size];

  for (int i = 0; i < size; i++) {
    A[i] = static_cast<float>(i);
  }

  float *d_A, *d_C;

  cudaMalloc(&d_A, size * sizeof(float));
  cudaMalloc(&d_C, size * sizeof(float));

  cudaMemcpy(d_A, A, size * sizeof(float), cudaMemcpyHostToDevice);

  int threadsPerBlock = size;
  int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;
  int shared_size = threadsPerBlock * sizeof(float);

  Pooling<<<blocksPerGrid, threadsPerBlock, shared_size>>>(d_A, d_C, size);

  cudaMemcpy(C, d_C, size * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_C);

  for (int i = 0; i < size; i++) {
    if (i >= 2) {
        assert(C[i] == A[i] + A[i-1] + A[i-2]);
    }
  }

  std::cout << "Pooling successful!" << std::endl;
  return 0;

}

Writing pooling.cu


In [ ]:
!nvcc pooling.cu -o pooling
!./pooling
!compute-sanitizer ./pooling


Pooling successful!
========= COMPUTE-SANITIZER
Pooling successful!
========= ERROR SUMMARY: 0 errors


In [ ]:
%%writefile dot_product.cu

#include <iostream>
#include <cassert>

__global__ void DotProduct(float* A, float* B, float* C, float size) {
  extern __shared__ float sharedMem[];
  int i = blockDim.x * blockIdx.x + threadIdx.x;
  int local_i = threadIdx.x;

  if (i < size) {
    sharedMem[local_i] = A[i] * B[i];
  }
  __syncthreads();

  if (local_i == 0) {
    int sum = 0;
    for (int k = 0; k < size; k++) {
      sum = sum + sharedMem[k];
    }
    C[0] = sum;
  }
}

int main() {

  const int size = 8;
  float A[size], B[size], C[1];

  for (int i = 0; i < size; i++) {
    A[i] = i;
  }

  for (int j = 0; j < size; j++) {
    B[j] = j;
  }

  float *d_A, *d_B, *d_C;

  cudaMalloc(&d_A, size * sizeof(float));
  cudaMalloc(&d_B, size * sizeof(float));
  cudaMalloc(&d_C, sizeof(float));

  cudaMemcpy(d_A, A, size * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, B, size * sizeof(float), cudaMemcpyHostToDevice);

  int threadsPerBlock = size;
  int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;
  int shared_size = threadsPerBlock * sizeof(float);

  DotProduct<<<blocksPerGrid, threadsPerBlock, shared_size>>>(d_A, d_B, d_C, size);

  cudaMemcpy(C, d_C, sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  int expected_dot_product = 0;
  for (int k = 0; k < size; k++) {
    expected_dot_product += A[k] * B[k];
  }
  assert(C[0] == expected_dot_product);

  std::cout << "Dot product successful!" << std::endl;

  return 0;
}


Overwriting dot_product.cu


In [ ]:
!nvcc dot_product.cu -o dot_product
!./dot_product
!compute-sanitizer ./dot_product

Dot product successful!
========= COMPUTE-SANITIZER
Dot product successful!
========= ERROR SUMMARY: 0 errors


In [ ]:
%%writefile 1d_conv.cu

#include <iostream>
#include <cassert>

const int TPB = 8;
const int MAX_CONV = 4;
const int TPB_MAX_CONV = TPB + MAX_CONV;

__global__ void Conv1D(float* A, float* B, float* C, int a_size, int b_size) {
  extern __shared__ float sharedMem[];

  float* shared_a = sharedMem;
  float* shared_b = sharedMem + TPB_MAX_CONV;

  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int local_i = threadIdx.x;

  if (i < a_size) {
    shared_a[local_i] = A[i];
  }

  if (local_i < b_size) {
    shared_b[local_i] = B[local_i];
  }
  else {
    int local_i2 = local_i - b_size;
    int i2 = i - b_size;
    if (i2 + TPB < a_size && local_i2 < b_size) {
      shared_a[local_i2 + TPB] = A[i2 + TPB];
    }
  }
  __syncthreads();

  int sum = 0;
  for (int j = 0; j < b_size; j++) {
    if (i + j < a_size) {
      sum += shared_a[local_i + j] * shared_b[j];
    }
  }

  if (i < a_size) {
    C[i] = sum;
  }
}

int main() {

  const int size = 5;

  float A[size], B[size-2], C[1];

  for (int i = 0; i < size; i++) {
    A[i] = i;
  }

  for (int j = 0; j < size-2; j++) {
    B[j] = j;
  }

  float *d_A, *d_B, *d_C;

  cudaMalloc(&d_A, size * sizeof(float));
  cudaMalloc(&d_B, (size - 2) * sizeof(float));
  cudaMalloc(&d_C, size * sizeof(float));

  cudaMemcpy(d_A, A, size * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, B, (size - 2) * sizeof(float), cudaMemcpyHostToDevice);

  int blocksPerGrid = (size + TPB - 1) / TPB;

  int shared_size_a = sizeof(float) * (TPB + MAX_CONV);
  int shared_size_b = sizeof(float) * MAX_CONV;

  Conv1D<<<blocksPerGrid, TPB, shared_size_a + shared_size_b>>>(
    d_A, d_B, d_C, size, (size - 2)
  );

  cudaMemcpy(C, d_C, size * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  float host_C[] = {5, 8, 11, 4, 0};

  for (int i = 0; i < size; i++) {
    assert(host_C[i] == C[i]);
  }

  std::cout << "1D Convolution successful!" << std::endl;

  return 0;

}

Overwriting 1d_conv.cu


In [ ]:
!nvcc 1d_conv.cu -o 1d_conv
!./1d_conv
!compute-sanitizer ./1d_conv

1D Convolution successful!
========= COMPUTE-SANITIZER
1D Convolution successful!
========= ERROR SUMMARY: 0 errors


In [73]:
%%writefile prefix_sum.cu

#include <iostream>
#include <cassert>
#include <math.h>

__global__ void PrefixSum(float* A, float* C, int size) {
  extern __shared__ float cache[];
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int local_i = threadIdx.x;

  if (i < size) {
    cache[local_i] = A[i];
  }
  __syncthreads();

  for (int p = 0; p < 3; p++) {
    int k = pow(2, p);
    if (local_i % (k * 2) == 0 && local_i + k < blockDim.x) {
      cache[local_i] += cache[local_i + k];
    }
    __syncthreads();
  }

  if (i < size) {
    C[i] = cache[local_i];
  }
}

int main() {

  const int size = 5;

  float A[size], C[size];

  for (int i = 0; i < size; i++) {
    A[i] = i;
  }

  float *d_A, *d_C;

  cudaMalloc(&d_A, size * sizeof(float));
  cudaMalloc(&d_C, size * sizeof(float));

  cudaMemcpy(d_A, A, size * sizeof(float), cudaMemcpyHostToDevice);

  int threadsPerBlock = size;
  int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;
  int shared_size = threadsPerBlock * sizeof(float);

  PrefixSum<<<blocksPerGrid, threadsPerBlock, shared_size>>>(
    d_A, d_C, size
  );

  cudaMemcpy(C, d_C, size * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_C);

  assert(C[0] == 10);

  std::cout << "Prefix sum successful!" << std::endl;

  return 0;
}

Overwriting prefix_sum.cu


In [74]:
!nvcc prefix_sum.cu -o prefix_sum
!./prefix_sum
!compute-sanitizer ./prefix_sum

Prefix sum successful!
========= COMPUTE-SANITIZER
Prefix sum successful!
========= ERROR SUMMARY: 0 errors


In [82]:
%%writefile axis_sum.cu

#include <iostream>
#include <cassert>
#include <math.h>

__global__ void AxisSum(float* A, float* C, int size) {
  extern __shared__ float cache[];
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int local_i = threadIdx.x;
  int batch = blockIdx.y;

  int flat_index = batch * size + i;

  if (i < size) {
    cache[local_i] = A[flat_index];
    __syncthreads();

    for (int p = 0; p < 3; p++) {
      int k = pow(2, p);
      if (local_i % (k * 2) == 0 && local_i + k < size) {
        cache[local_i] += cache[local_i + k];
      }
      __syncthreads();
    }

    if (i < size) {
      C[batch] = cache[local_i];
    }
  }
}

int main() {

  const int size = 5;
  const int numBatches = 1;

  float A[size * numBatches], C[numBatches];

  for (int j = 0; j < numBatches; j++) {
    for (int i = 0; i < size; i++) {
      A[j * size + i] = i;
    }
  }

  float *d_A, *d_C;

  cudaMalloc(&d_A, size * numBatches * sizeof(float));
  cudaMalloc(&d_C, numBatches * sizeof(float));

  cudaMemcpy(d_A, A, size * numBatches * sizeof(float), cudaMemcpyHostToDevice);

  dim3 threadsPerBlock(size, 1);
  dim3 blocksPerGrid(1, numBatches);
  int shared_size = threadsPerBlock.x * sizeof(float);

  AxisSum<<<blocksPerGrid, threadsPerBlock, shared_size>>>(d_A, d_C, size);

  cudaMemcpy(C, d_C, numBatches * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_C);

  assert(C[0] == 10);

  std::cout << "Axis sum successful!" << std::endl;

  return 0;
}

Overwriting axis_sum.cu


In [83]:
!nvcc axis_sum.cu -o axis_sum
!./axis_sum
!compute-sanitizer ./axis_sum

Axis sum successful!
========= COMPUTE-SANITIZER
Axis sum successful!
========= ERROR SUMMARY: 0 errors


In [10]:
%%writefile matmul.cu

#include <iostream>
#include <cassert>

const int TPB = 3;

__global__ void Matmul(float* A, float* B, float* C, int size) {
  extern __shared__ float sharedMem[];
  float* a_shared = sharedMem;
  float* b_shared = sharedMem + (TPB * TPB);

  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  int local_i = threadIdx.x;
  int local_j = threadIdx.y;

  float sum = 0;
  for (int k = 0; k < size; k += TPB) {
    if (i < size && k + local_j < size) {
      a_shared[local_i * TPB + local_j] = A[i * size + k + local_j];
    }
    if (j < size && k + local_i < size) {
      b_shared[local_i * TPB + local_j] = B[(local_i + k) * size + j];
    }
    __syncthreads();

    for (int local_k = 0; local_k < TPB; local_k++) {
      if (k + local_k < size) {
        sum += a_shared[local_i * TPB + local_k] * b_shared[local_k * TPB + local_j];
      }
    }
  }
  if (i < size && j < size) {
    C[i * size + j] = sum;
  }
}

int main() {

  const int size = 2;
  float A[size][size], B[size][size], C[size][size];

  for (int i = 0; i < size; i++) {
    for (int j = 0; j < size; j++) {
      A[i][j] = i * j;
      B[i][j] = i + j;
    }
  }

  float *d_A, *d_B, *d_C;
  cudaMalloc(&d_A, (size * size) * sizeof(float));
  cudaMalloc(&d_B, (size * size) * sizeof(float));
  cudaMalloc(&d_C, (size * size) * sizeof(float));

  cudaMemcpy(d_A, A, (size * size) * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, B, (size * size) * sizeof(float), cudaMemcpyHostToDevice);

  int BpG = (size + TPB - 1) / TPB;
  dim3 blocksPerGrid(BpG, BpG);
  dim3 threadsPerBlock(TPB, TPB);
  int sharedMemSize = 2 * (TPB * TPB) * sizeof(float);

  Matmul<<<blocksPerGrid, threadsPerBlock, sharedMemSize>>>(d_A, d_B, d_C, size);

  cudaMemcpy(C, d_C, (size * size) * sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  assert(C[0][0] == 0);
  assert(C[0][1] == 0);
  assert(C[1][0] == 1);
  assert(C[1][1] == 2);

  std::cout << "Matrix multiplication successful!" << std::endl;

  return 0;
}



Overwriting matmul.cu


In [11]:
!nvcc matmul.cu -o matmul
!./matmul
!compute-sanitizer ./matmul

Matrix multiplication successful!
========= COMPUTE-SANITIZER
Matrix multiplication successful!
========= ERROR SUMMARY: 0 errors
